In [1]:
# imports required for the example
import pandas as pd

# sklearn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# category encoders package (see: https://contrib.scikit-learn.org/category_encoders/index.html)
from category_encoders.leave_one_out import LeaveOneOutEncoder

# optuna + visulization
import optuna
from optuna.integration.mlflow import MLflowCallback
import plotly.graph_objects as go
import ipywidgets as ipyw

# autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

# to manage the should_run_async(code) warning in relation to ipykernel and IPython
import warnings
warnings.filterwarnings('ignore')

# Background

Data science should be an enjoyable process focused on delivering insights and real benefits. However, this enjoyable process can sometimes get lost in tools and processes. Nowadays it is important for an applied data scientist to develop their own toolbox by selecting tools and software to cover things like data/code versioning, reproducible model development, experiment tracking, and model inspection - to name a few!  

The purpose of this article is to briefly touch upon reproducible model development and experiment tracking through a worked example for tuning a Random Forest classifier using [Optuna](https://optuna.readthedocs.io/en/stable/index.html) and [mlflow](https://www.mlflow.org/docs/latest/index.html).

## Loading the heart disease data and creating a pre-processing pipeline

This example uses the heart disease data available in the [UCI ML archive](https://archive.ics.uci.edu/ml/datasets/heart+Disease). In my previous work I describe some processing done to create a semi-cleaned dataset: [developing a model for heart disease prediction using pycaret](https://towardsdatascience.com/developing-a-model-for-heart-disease-prediction-using-pycaret-9cdf03a66f42). In the csv file provided for this short worked example, I have additionally converted any nominal features with two values into a single binary feature where `_x` at the end of the feature name indicates the level represented by a value of 1. The target is called `hd_yes` and we want to tune a Random Forest classifier to predict heart disease.

First let's load in the dataset and have a quick look.

In [2]:
hd_df = pd.read_csv('heart_disease_processed.csv')
hd_df.head(10)

,age,sex_male,chest_pain,resting_bp,serum_cholesterol,high_fasting_blood_sugar_yes,resting_ecg_not_normal,maximum_hr,exercise_induced_angina_yes,ST_depression_exercise_vs_rest,peak_exercise_ST_segment_slope_flat_downsloping,num_affected_major_vessels,thallium_stress_test_bf_defect,hd_yes
0,63,1,anginal_pain,145,233.0,1,1,150,0,2.3,1,0.0,1.0,0
1,67,1,asymptomatic,160,286.0,0,1,108,1,1.5,1,3.0,0.0,1
2,67,1,asymptomatic,120,229.0,0,1,129,1,2.6,1,2.0,1.0,1
3,37,1,non_anginal_pain,130,250.0,0,0,187,0,3.5,1,0.0,0.0,0
4,41,0,anginal_pain,130,204.0,0,1,172,0,1.4,0,0.0,0.0,0
5,56,1,anginal_pain,120,236.0,0,0,178,0,0.8,0,0.0,0.0,0
6,62,0,asymptomatic,140,268.0,0,1,160,0,3.6,1,2.0,0.0,1
7,57,0,asymptomatic,120,354.0,0,0,163,1,0.6,0,0.0,0.0,0
8,63,1,asymptomatic,130,254.0,0,1,147,0,1.4,1,1.0,1.0,1
9,53,1,asymptomatic,140,203.0,1,1,155,1,3.1,1,0.0,1.0,1


Create the feature matrix (**X**) and the target vector (**y**).

In [3]:
X_full = hd_df.drop(columns=['hd_yes'])
y_full = hd_df['hd_yes'].ravel()

Next we will create a pre-processing pipeline that will process 3 sets of features in slightly different ways.

**(1)** For numeric features we will impute any missing values using the median and then apply scaling

In [4]:
numeric_features = ['age', 'resting_bp', 'serum_cholesterol', 'maximum_hr', 'ST_depression_exercise_vs_rest',
                   'num_affected_major_vessels']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

**(2)** For binary features we will impute missing values using the mode

In [5]:
binary_features = ['sex_male', 'high_fasting_blood_sugar_yes', 'resting_ecg_not_normal', 'exercise_induced_angina_yes',
                  'peak_exercise_ST_segment_slope_flat_downsloping', 'thallium_stress_test_bf_defect']

binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

**(3)** For the three-level categorical feature chest pain, missing values were treated as a separate category and [leave-one-out target encoding](https://contrib.scikit-learn.org/category_encoders/leaveoneout.html) applied - not really recommended for a categorical feature with three levels, but it does provide an example of how you might want to handle different category related features in different ways in a preprocessing pipeline.

In [6]:
categorical_features = ['chest_pain']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('loo_encoder', LeaveOneOutEncoder())
])

Finally, all the different feature processing steps are wrapped up into a single processing pipeline.

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('bin', binary_transformer, binary_features),        
        ('cat', categorical_transformer, categorical_features)
])

## Using Optuna and MLflow

With our feature matrix, target vector and preprocessing pipeline ready to go, we can now tune a Random Forest classifier to predict heart disease. Note for the purpose of this demonstration I am going to forgo the use of a hold-out test set. To do the hyper-parameter optimization (model development) we will use Optuna and for experiment tracking one of its [newer features](https://medium.com/optuna/optuna-v2-3165e3f1fc2): mlflow integration (see: [mlflow callback in Optuna](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.MLflowCallback.html?highlight=mlflow) and [mlflow tracking](https://www.mlflow.org/docs/latest/tracking.html)).

First we define the objective function to optimize with Optuna, which has three main components:

1. **Defining the space of hyper-parameters and values to optimize**: in our case we focus on six hyper-parameters (n_estimators, max_depth, max_features, bootstrap, min_samples_split, min_samples_split)
2. **The Machine Learning pipeline**: we have a preprocessing pipeline plus a Random Forest classifier
3. **The calculation of the metric to optimize**: we use the 5-fold CV average ROC AUC

In [8]:
def objective(trial):
    
    # parameters to optimize
    n_estimators = trial.suggest_int('n_estimators', 10, 500, 10)
    max_depth = trial.suggest_int('max_depth', 1, 32)
    max_features = trial.suggest_int('max_features', 2, 13)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    min_samples_split = trial.suggest_discrete_uniform('min_samples_split', 0.1, 1.0, 0.1)
    min_samples_leaf = trial.suggest_discrete_uniform('min_samples_leaf', 0.1, 0.5, 0.1)
    
    # our ML pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', RandomForestClassifier(n_estimators=n_estimators,
                                                                max_depth=max_depth,
                                                                max_features=max_features,
                                                                bootstrap=bootstrap,
                                                                min_samples_split=min_samples_split,
                                                                min_samples_leaf=min_samples_leaf,
                                                                random_state=25
                                                                ))])
    
    # return average 5-fold CV ROC AUC
    return cross_val_score(clf, X_full, y_full, scoring='roc_auc', n_jobs=-1, cv=5).mean()

Next we define a callback to mlflow, one of the new (and experimental!) features in Optuna 2.0 in order to track our classifier hyper-parameter tuning performed with Optuna.

In [9]:
mlflow_cb = MLflowCallback(
    tracking_uri='mlruns',
    metric_name='ROC AUC'
)

Next we instantiate the Optuna study object to maximize our chosen metric which in this case is the ROC AUC. We also apply [Hyperband pruning](https://optuna.readthedocs.io/en/latest/reference/generated/optuna.pruners.HyperbandPruner.html), which helps find optimums in shorter times by stopping unpromising trials early. Note that this is typically of more benefit for more intensive and difficult optimization such as in deep learning.

In [10]:
hd_study = optuna.create_study(
    study_name='heart_disease_prediction',
    direction='maximize',
    pruner=optuna.pruners.HyperbandPruner(max_resource="auto")
)

Finally, we run the optimization using 200 trials. It may take a couple of minutes to run.

In [11]:
hd_study.optimize(objective, n_trials=200, callbacks=[mlflow_cb])

[I 2020-09-03 12:34:08,446] Trial 0 finished with value: 0.901746632996633 and parameters: {'n_estimators': 270, 'max_depth': 7, 'max_features': 5, 'bootstrap': False, 'min_samples_split': 0.5, 'min_samples_leaf': 0.2}. Best is trial 0 with value: 0.901746632996633.


INFO: 'heart_disease_prediction' does not exist. Creating a new experiment


[I 2020-09-03 12:34:10,600] Trial 1 finished with value: 0.5 and parameters: {'n_estimators': 30, 'max_depth': 26, 'max_features': 7, 'bootstrap': True, 'min_samples_split': 0.8, 'min_samples_leaf': 0.4}. Best is trial 0 with value: 0.901746632996633.
[I 2020-09-03 12:34:11,465] Trial 2 finished with value: 0.5621212121212121 and parameters: {'n_estimators': 380, 'max_depth': 12, 'max_features': 9, 'bootstrap': False, 'min_samples_split': 0.6, 'min_samples_leaf': 0.5}. Best is trial 0 with value: 0.901746632996633.
[I 2020-09-03 12:34:12,269] Trial 3 finished with value: 0.8344832251082253 and parameters: {'n_estimators': 370, 'max_depth': 21, 'max_features': 12, 'bootstrap': False, 'min_samples_split': 0.9, 'min_samples_leaf': 0.2}. Best is trial 0 with value: 0.901746632996633.
[I 2020-09-03 12:34:12,662] Trial 4 finished with value: 0.87445273669232 and parameters: {'n_estimators': 130, 'max_depth': 30, 'max_features': 7, 'bootstrap': False, 'min_samples_split': 0.6, 'min_samples_le

[I 2020-09-03 12:34:40,258] Trial 30 finished with value: 0.9090042287958955 and parameters: {'n_estimators': 350, 'max_depth': 6, 'max_features': 5, 'bootstrap': False, 'min_samples_split': 0.2, 'min_samples_leaf': 0.1}. Best is trial 15 with value: 0.9145277176527177.
[I 2020-09-03 12:34:41,278] Trial 31 finished with value: 0.9134286816578484 and parameters: {'n_estimators': 310, 'max_depth': 8, 'max_features': 4, 'bootstrap': False, 'min_samples_split': 0.1, 'min_samples_leaf': 0.1}. Best is trial 15 with value: 0.9145277176527177.
[I 2020-09-03 12:34:42,470] Trial 32 finished with value: 0.9125117744909412 and parameters: {'n_estimators': 300, 'max_depth': 9, 'max_features': 3, 'bootstrap': False, 'min_samples_split': 0.2, 'min_samples_leaf': 0.1}. Best is trial 15 with value: 0.9145277176527177.
[I 2020-09-03 12:34:43,437] Trial 33 finished with value: 0.9136531485489818 and parameters: {'n_estimators': 270, 'max_depth': 12, 'max_features': 4, 'bootstrap': False, 'min_samples_spl

[I 2020-09-03 12:35:11,067] Trial 59 finished with value: 0.911304137606221 and parameters: {'n_estimators': 490, 'max_depth': 14, 'max_features': 2, 'bootstrap': False, 'min_samples_split': 0.9, 'min_samples_leaf': 0.2}. Best is trial 15 with value: 0.9145277176527177.
[I 2020-09-03 12:35:12,350] Trial 60 finished with value: 0.9125303130511464 and parameters: {'n_estimators': 450, 'max_depth': 20, 'max_features': 2, 'bootstrap': False, 'min_samples_split': 0.2, 'min_samples_leaf': 0.2}. Best is trial 15 with value: 0.9145277176527177.
[I 2020-09-03 12:35:13,853] Trial 61 finished with value: 0.9142649210357543 and parameters: {'n_estimators': 390, 'max_depth': 12, 'max_features': 3, 'bootstrap': False, 'min_samples_split': 0.2, 'min_samples_leaf': 0.1}. Best is trial 15 with value: 0.9145277176527177.
[I 2020-09-03 12:35:14,919] Trial 62 finished with value: 0.9149303050344717 and parameters: {'n_estimators': 400, 'max_depth': 12, 'max_features': 3, 'bootstrap': False, 'min_samples_s

[I 2020-09-03 12:36:02,664] Trial 89 finished with value: 0.9127049262465927 and parameters: {'n_estimators': 410, 'max_depth': 29, 'max_features': 5, 'bootstrap': True, 'min_samples_split': 0.1, 'min_samples_leaf': 0.1}. Best is trial 77 with value: 0.9151344797178131.
[I 2020-09-03 12:36:04,319] Trial 90 finished with value: 0.9063745089786757 and parameters: {'n_estimators': 430, 'max_depth': 19, 'max_features': 6, 'bootstrap': True, 'min_samples_split': 0.30000000000000004, 'min_samples_leaf': 0.1}. Best is trial 77 with value: 0.9151344797178131.
[I 2020-09-03 12:36:06,661] Trial 91 finished with value: 0.9151344797178131 and parameters: {'n_estimators': 500, 'max_depth': 21, 'max_features': 4, 'bootstrap': True, 'min_samples_split': 0.2, 'min_samples_leaf': 0.1}. Best is trial 77 with value: 0.9151344797178131.
[I 2020-09-03 12:36:08,568] Trial 92 finished with value: 0.9149137706429373 and parameters: {'n_estimators': 490, 'max_depth': 22, 'max_features': 4, 'bootstrap': True, '

[I 2020-09-03 12:36:51,752] Trial 118 finished with value: 0.9155821608946608 and parameters: {'n_estimators': 350, 'max_depth': 19, 'max_features': 4, 'bootstrap': True, 'min_samples_split': 0.2, 'min_samples_leaf': 0.1}. Best is trial 114 with value: 0.9166939734648067.
[I 2020-09-03 12:36:53,450] Trial 119 finished with value: 0.9136416245791246 and parameters: {'n_estimators': 330, 'max_depth': 19, 'max_features': 5, 'bootstrap': True, 'min_samples_split': 0.2, 'min_samples_leaf': 0.1}. Best is trial 114 with value: 0.9166939734648067.
[I 2020-09-03 12:36:54,825] Trial 120 finished with value: 0.9166939734648067 and parameters: {'n_estimators': 340, 'max_depth': 18, 'max_features': 4, 'bootstrap': True, 'min_samples_split': 0.2, 'min_samples_leaf': 0.1}. Best is trial 114 with value: 0.9166939734648067.
[I 2020-09-03 12:36:56,470] Trial 121 finished with value: 0.9155821608946608 and parameters: {'n_estimators': 350, 'max_depth': 18, 'max_features': 4, 'bootstrap': True, 'min_sampl

[I 2020-09-03 12:37:34,545] Trial 147 finished with value: 0.9166939734648067 and parameters: {'n_estimators': 340, 'max_depth': 15, 'max_features': 4, 'bootstrap': True, 'min_samples_split': 0.2, 'min_samples_leaf': 0.1}. Best is trial 114 with value: 0.9166939734648067.
[I 2020-09-03 12:37:35,998] Trial 148 finished with value: 0.9124854697771363 and parameters: {'n_estimators': 340, 'max_depth': 14, 'max_features': 3, 'bootstrap': True, 'min_samples_split': 0.2, 'min_samples_leaf': 0.1}. Best is trial 114 with value: 0.9166939734648067.
[I 2020-09-03 12:37:37,456] Trial 149 finished with value: 0.9162826178451178 and parameters: {'n_estimators': 290, 'max_depth': 18, 'max_features': 4, 'bootstrap': True, 'min_samples_split': 0.2, 'min_samples_leaf': 0.1}. Best is trial 114 with value: 0.9166939734648067.
[I 2020-09-03 12:37:38,951] Trial 150 finished with value: 0.5 and parameters: {'n_estimators': 360, 'max_depth': 19, 'max_features': 3, 'bootstrap': True, 'min_samples_split': 1.0,

[I 2020-09-03 12:38:18,010] Trial 177 finished with value: 0.9124854697771363 and parameters: {'n_estimators': 340, 'max_depth': 15, 'max_features': 3, 'bootstrap': True, 'min_samples_split': 0.2, 'min_samples_leaf': 0.1}. Best is trial 114 with value: 0.9166939734648067.
[I 2020-09-03 12:38:19,250] Trial 178 finished with value: 0.9151369849286516 and parameters: {'n_estimators': 360, 'max_depth': 14, 'max_features': 4, 'bootstrap': True, 'min_samples_split': 0.2, 'min_samples_leaf': 0.1}. Best is trial 114 with value: 0.9166939734648067.
[I 2020-09-03 12:38:21,199] Trial 179 finished with value: 0.9166939734648067 and parameters: {'n_estimators': 340, 'max_depth': 15, 'max_features': 4, 'bootstrap': True, 'min_samples_split': 0.1, 'min_samples_leaf': 0.1}. Best is trial 114 with value: 0.9166939734648067.
[I 2020-09-03 12:38:23,054] Trial 180 finished with value: 0.9124977453102453 and parameters: {'n_estimators': 380, 'max_depth': 14, 'max_features': 5, 'bootstrap': True, 'min_sampl

The mlflow logged experiment including assessed hyper-parameter configurations for the Random Forest classifier (Optuna study/trials), are stored in a folder called mlruns. To view these results using the mlflow user interface, do the following:

1. Open a shell (for example a Windows PowerShell) in the directory containing the mlruns folder
2. Activate the virtual environment used for this notebook, i.e., `conda activate optuna_env`
3. Run `mlflow ui`
4. Navigate to the localhost address provided, something like: `http://kubernetes.docker.internal:5000/`

You should see **Figure 1**, and you can sift through the experiment trials. I deselected the user, source and version columns, as well as the tags section to simplify the view. I also sorted the results by ROC AUC. The top hyper-parameter configurations provided a ROC AUC of 0.917, where max_depth was in the range 14 to 16, max_features was 4, min_samples_leaf was 0.1, min_samples_split was 0.2, and n_estimators was 340 (note these results may differ slightly for you if you re-run the notebook).

![](mlflow_ui_snip.PNG)

**Figure 1**: mlflow summary for Optuna HPO of our heart disease random Forest classifier

Optuna also has some great built in visualization tools for summarizing your experiments as well. Here we look at hyper-parameter importance first and display the optimization history. The hyper-parameter importance is calculated using a functional ANOVA approach and for more information please see the article by [Hutter, Hoos and Leyton-Brown 2014](http://proceedings.mlr.press/v32/hutter14.pdf).

In [15]:
# Lets create a panel with a couple of optuna built-in visulizations: note the update_layout() used here is just to ensure 
# everything displays without being cutoff.

fig1 = optuna.visualization.plot_optimization_history(hd_study)
fw1 = go.FigureWidget(fig1)
fw1.update_layout(width=610, margin=dict(r=250))

fig2 = optuna.visualization.plot_param_importances(hd_study)
fw2 = go.FigureWidget(fig2)
fw2.update_layout(margin=dict(r=100))

fig_subplots=ipyw.HBox([fw2, fw1])
fig_subplots

    'data': [{'cliponaxis': False,
              'hovertemplate': [max_features …

# Summary

Optuna has come along way since its inception and version 2.0 released in July this year has some fantastic additions, one of which we touched upon in this short article: the mlflow callback. Perhaps as expected the tuning worked great and the summaries available via the in-built visualizations in Optuna and using the mlflow UI made this a lot of fun. It all ran smoothly even with a decent preprocessing pipeline and half a dozen hyper-parameters to tune.